In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, nsolve
from sympy import cosh, sin , sinh, cos,exp
from scipy.special import sph_harm
from scipy.optimize import fsolve
from scipy.integrate import quad
import numba
from decimal import Decimal, getcontext
getcontext().prec = 50  # 设置精度

In [ ]:
e2=1.43997 ; hbarc=197.3269718 ; amu=931.49432
zp=55 ; mp=112*amu-46.41
z1=54 ; m1=111*amu-54.52
z2=1 ; m2=1*amu+7.288971064
Q=mp-m1-m2
mu=m1*m2/(m1+m2)

In [ ]:
def Vn(V0,r0,a0,beta2,beta4,beta2_tilde,beta4_tilde,r,theta):
    Y20=sph_harm(0, 2, 0, theta)  #sph_harm(m,l,phi,theta)
    Y40=sph_harm(0, 4, 0, theta)
    r0_theta=r0*(1+beta2*Y20+beta4*Y40)         #deformation

    delta_theta=1e-6
    dY20=(sph_harm(0, 2, 0, theta+delta_theta)-sph_harm(0, 2, 0, theta))/(delta_theta)
    dY40=(sph_harm(0, 4, 0, theta+delta_theta)-sph_harm(0, 4, 0, theta))/(delta_theta)
    dr0_theta=r0*(beta2*dY20+beta4*dY40)       

# polarization
    a0_theta=a0*np.sqrt(1+(1/r0*dr0_theta)**2)*(1+beta2_tilde*Y20+beta4_tilde*Y40) 

    return -V0/(1+np.exp((r-r0_theta)/a0_theta))

In [ ]:

# 参数设置
V0 = 50
r0 = 5
a0 = 0.5
beta2 = 0.2
beta4 = 0.1
beta2_tilde = 0.05
beta4_tilde = 0.02

# 创建径向距离数组
r_values = np.linspace(0.1, 10, 200)

# 90度（π/2 弧度）
theta_90 = np.pi / 2

# 计算90度下的势能
Vn_90 = [Vn(V0, r0, a0, beta2, beta4, beta2_tilde, beta4_tilde, r, theta_90) for r in r_values]
Vn_0 = [Vn(V0, r0, a0, beta2, beta4, beta2_tilde, beta4_tilde, r, 0) for r in r_values]
# 绘制图形
plt.figure(figsize=(10, 6))
plt.plot(r_values, Vn_90, 'b-', linewidth=2, label='θ = 90°')
plt.plot(r_values, Vn_0, 'r-', linewidth=2, label='θ = 0°')

plt.xlabel(' r', fontsize=12)
plt.ylabel('Vn', fontsize=12)

plt.grid(True, alpha=0.3)
plt.xlim(0, 10)
plt.tight_layout()
plt.legend(fontsize=12)
plt.show()